# Database APIs for ASER

Firstly, install the necessay packages

In [ ]:
!pip install -r requirements.txt 

Then, install the aser package.

In [ ]:
!python setup.py develop

Now, we can import the necessay APIs to access the database

In [12]:
from aser.database.db_API import KG_Connection
from aser.database.db_API import generate_event_id, generate_relation_id, generate_id

We provide three different modes: *insert*, *cache*, and *memory*.
* *insert* mode supports the insert operation.
* *cache* mode load necessary metadata in the memory
* *memory* mode loads the whole data in the memory

However, it still takes a lot of time, and costs a lot memory due to the large size.
We use the *cache* mode to build a DB connection.

In [6]:
kg_conn = KG_Connection(db_path=r'data/database/core/KG_v0.1.0.db', mode='cache')

In [7]:
print('SIZE:', 'eventualities: ', len(kg_conn.event_id_set), 'relations:', len(kg_conn.relation_id_set))

SIZE: eventualities:  27565673 relations: 8834257


Noted that eventualites extracted from differnent sentences are identified as the unique one in the database if their words are exactly the same, and relations between two eventualities are identified as the unique one distribution in the database. 

We can print the attributes of eventualites and relations.

In [8]:
print(list(zip(kg_conn.event_columns, kg_conn.event_column_types)))

[('_id', 'PRIMARY KEY'), ('verbs', 'TEXT'), ('skeleton_words_clean', 'TEXT'), ('skeleton_words', 'TEXT'), ('words', 'TEXT'), ('pattern', 'TEXT'), ('frequency', 'REAL')]


In [9]:
print(list(zip(kg_conn.relation_columns, kg_conn.relation_column_types)))

[('_id', 'PRIMARY KEY'), ('event1_id', 'TEXT'), ('event2_id', 'TEXT'), ('Precedence', 'REAL'), ('Succession', 'REAL'), ('Synchronous', 'REAL'), ('Reason', 'REAL'), ('Result', 'REAL'), ('Condition', 'REAL'), ('Contrast', 'REAL'), ('Concession', 'REAL'), ('Conjunction', 'REAL'), ('Instantiation', 'REAL'), ('Restatement', 'REAL'), ('ChosenAlternative', 'REAL'), ('Alternative', 'REAL'), ('Exception', 'REAL'), ('Co_Occurrence', 'REAL')]


We can retrieve eventualities from the database. 
The extractor is not released at present, we just use simple words to check.

In [13]:
kg_conn.get_exact_match_event(generate_id('i learn python'))

OrderedDict([('_id', 'ce3407c430d7757367196b4222d63bead2cea2d7'),
             ('verbs', 'learn'),
             ('skeleton_words_clean', 'learn python'),
             ('skeleton_words', 'i learn python'),
             ('words', 'i learn python'),
             ('pattern', 's-v-o'),
             ('frequency', 12.0)])

Relations can also be retrieved.

In [16]:
kg_conn.get_exact_match_relation([generate_id('i be tired'), generate_id('i sleep')])

OrderedDict([('_id', '4ac9d37cb326ad778c13ecf36109a1b411513e4b'),
             ('event1_id', '4394f2e2056678dd4939e11bb52c4f4a03411fa5'),
             ('event2_id', '1b4d7b0d2131a548c9a607dd0b62fc410172ceac'),
             ('Precedence', 0.0),
             ('Succession', 0.0),
             ('Synchronous', 0.0),
             ('Reason', 0.0),
             ('Result', 0.9970114231109619),
             ('Condition', 0.0),
             ('Contrast', 0.0),
             ('Concession', 0.0),
             ('Conjunction', 0.0),
             ('Instantiation', 0.0),
             ('Restatement', 0.0),
             ('ChosenAlternative', 0.0),
             ('Alternative', 0.0),
             ('Exception', 0.0),
             ('Co_Occurrence', 4.0)])